In [ ]:
import tensorflow as tf
from google.colab import drive

drive.mount('/content/drive')

# Load your Keras model
keras_model_path = '/content/drive/MyDrive/neww_dr_skin_model.h5'
keras_model = tf.keras.models.load_model(keras_model_path)

# Convert the model to TensorFlow Lite with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

# Save the quantized TensorFlow Lite model
tflite_quantized_model_path = '/content/drive/MyDrive/dr-skin-new--.tflite'

with open(tflite_quantized_model_path, "wb") as f:
    f.write(tflite_quantized_model)

Mounted at /content/drive


In [ ]:
import numpy as np
import cv2

# Load the TFLite model and allocate tensors
tflite_quantized_model_path = "/content/drive/MyDrive/dr-skin-new--.tflite"
interpreter = tf.lite.Interpreter(model_path=tflite_quantized_model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Define a function to run inference
def run_inference(interpreter, input_data):
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    return interpreter.get_tensor(output_details[0]['index'])

# Load your input image
image_path = "/content/drive/MyDrive/mwww.jpeg"
IMAGE_SIZE = 100
img = cv2.imread(image_path, cv2.IMREAD_COLOR)
if img is None:
    raise ValueError("Could not read the image file.")
if img.shape[:2] != (IMAGE_SIZE, IMAGE_SIZE):
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))

# Normalize the input image
input_data = np.array(img, dtype=np.float32) / 255.0
input_data = input_data.reshape((1,) + input_data.shape)

# Run inference on the input image
prediction = run_inference(interpreter, input_data)
predicted_class = np.argmax(prediction)

# Map the predicted class index to its corresponding name
lesion_type_dict = {
    0: 'Melanocytic nevi',
    1: 'Melanoma',
    2: 'Benign keratosis-like lesions',
    3: 'Basal cell carcinoma',
    4: 'Actinic keratoses',
    5: 'Vascular lesions',
    6: 'Dermatofibroma'
}

predicted_class_name = lesion_type_dict[predicted_class]

print(f"Predicted class: {predicted_class_name}")

ValueError: ignored